In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import itertools
from functools import total_ordering
from numpy.random import default_rng

COUL_FACTOR = 8.988e9 #prefactor in numberator of coulomb force

In [ ]:
@total_ordering #allows us to only implement lt and eq, imply gt, etc.
class Disk:
    def __init__(self, x:np.ndarray, v:np.ndarray, mass:float=3, 
                 radius:float=.5, charge:float=1.602e-3):
        self.m = mass
        self.r = radius
        self.x = np.asarray(x) #x example: array([2, 4])
        self.v = np.asarray(v) #v example: array([1.2, -2])
        self.nDim = len(self.x)
        if self.nDim != len(self.v):
            raise Exception("Dimensions of velocity and position lists do not match.")
        self.q = charge
    
    def forceFrom(self, other, L):
        #calculates the force vector on self because of other
        r = self.x - other.x
        r[r > L/2] -= L
        r[r < -L/2] += L
        return (COUL_FACTOR * self.q * other.q) * r / (np.linalg.norm(r)**3)
    
    def advance(self, dt:float, L, F=0):
        # apply old velocity (update position)
        self.x += self.v * dt
        self.x = self.x % L
        
        # apply force (update velocity)
        self.v += F * (dt / self.m)
    
    #allowing comparisons between disk
    def __lt__(self, other):
        return self.x[0] < other.x[0]
    
    def __eq__(self, other):
        return self.x[0] == other.x[0]
    
    @property
    def speed(self):
        return np.linalg.norm(self.v)
    
    @property
    def KE(self):
        return (1/2)*self.m*(self.speed**2)


In [36]:
class Expt:
    def __init__(self, particles, dt:float=0.1, t_0:float=0, 
                 tmax:float=15, L:float=200, animSpeed:float=1,
                updateGraphsEvery:int=5):
        # pPositions example: [ [1, 3], [2, 2] ]: two particles, at (1,3) and (2,2)
        
        # set time variables
        self.t = t_0
        self.tmax = tmax
        self.dt = dt
        self.animSpeed = animSpeed
        self.updateGraphsEvery = updateGraphsEvery
        
        # make the particle list
        self.particles = np.asarray(particles)
        self.numParticles = self.particles.size
        self.nDim = self.particles[0].nDim 
        
        #make the box bounds
        self.L = L
    
    def forceBetween(self, p1, p2):
        #given two particle IDs/indices, returns the force between them
        if p1 == p2: # in future, maybe change to if p1.friendsWith(p2) and
                     # have particles have a friends list who they don't push
            return 0
        else:
            return self.particles[p1].forceFrom(self.particles[p2], self.L)
    
    def nextFrame(self):
        
        #calculate forces in advance
        forces = np.zeros_like(self.particlePositions)
        
        #calculate all particles' interactions
        for p1 in range(self.numParticles):
            for p2 in range(self.numParticles):
                if p1 != p2:
                    forces[p1] += self.forceBetween(p1, p2)
        
        #move particles and apply forces afterwards, to allow simultaneity
        forceIter = iter(forces)
        for p in self.particles:
            p.advance(self.dt, self.L, next(forceIter))
        self.t += self.dt
    
    @property
    def totalKE(self):
        return sum(p.KE for p in self.particles)
    
    @property
    def avgKE(self):
        return self.totalKE / self.numParticles
    
    @property
    def particlePositions(self):
        return np.array([p.x for p in self.particles])
    
    def showAnimation(self, addlTitle=""):
        fig, ax = plt.subplots()              # create the figure
        ax.set_xlim(0,self.L)              # and adjust axes limits and labels
        ax.set_ylim(0,self.L)
        ax.set_title(self.t)
        xvar = np.linspace(0.1,self.L-0.1,self.numParticles)
        points, = ax.plot(xvar,np.ones_like(xvar), 'o')
        self.updatectr = 0
        
        def frame(_):
            points.set_data(np.transpose(self.particlePositions))
            title = ax.set_title(addlTitle + "t = {:0.2f}".format(self.t))
            self.nextFrame()
            
            #progress bar! (?)
            if self.c % self.updateGraphsEvery == 0:
                x = int(np.floor(8*self.t/self.tmax)+1)
                print ("[" + "FINISHED!"[:x] + "_________"[x:] + "]", end="\r")
                #TODO: make & update graphs
            self.c += 1
            
            return points, title 

        #somewhat glitchy display. IDK how best to fix.
        ani = FuncAnimation(fig, frame, np.arange(self.t,self.tmax,self.dt), 
                            interval=self.dt*1000/self.animSpeed, blit=True,
                           repeat=False)
        print("bouta save animation...")
        ani.save("particleAnimation.gif")
        plt.close()
    
    #idea: makeCopy() function that makes an identical experiment - might be useful to
    #let us go to further times or something? idk
    
    def getKEs(self):
        """
        Returns a list of the particles' potential energies in the current frame.
        """
        return [p.KE for p in self.particles]

    def showAnimation1(self, addlTitle=""):
        """
        Modified version of showAnimation that creates a histogram of the particles' kinetic energies.
        """
        
        fig, (ax1, ax2) = plt.subplots(1, 2)              # create the figure
        ax1.set_xlim(0,self.L)              # and adjust axes limits and labels
        ax1.set_ylim(0,self.L)
        ax1.set_title(self.t)
        #TODO: change ax2 axes, title
        HIST_BINS = np.linspace(0, 1000, 100)
        self.updatectr = 0
        
        xvar = np.linspace(0.1,self.L-0.1,self.numParticles) #temporary variable
        points, = ax1.plot(xvar,np.ones_like(xvar), 'o')

        _, _, bar_container = ax2.hist(self.getKEs(), HIST_BINS, lw=1,
                              ec="yellow", fc="green", alpha=0.5)
        
        def frame(_):
            #animate
            points.set_data(np.transpose(self.particlePositions))
            title = ax1.set_title(addlTitle + "t = {:0.2f}".format(self.t))
            
            if self.updatectr % self.updateGraphsEvery == 0:
                n, _ = np.histogram(self.getKEs(), HIST_BINS)
                for count, rect in zip(n, bar_container.patches):
                    rect.set_height(count)
            
            
            #update
            self.nextFrame()
            
            #progress bar
            x = int(np.floor(32*self.t/self.tmax)+1)
            print ("[" + "████████████████████████████████"[:x] + "▄"*(x<32) + "▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁"[x:] + "]  ", end="\r")
            
            self.updatectr += 1
            return points, bar_container.patches, title 

        #somewhat glitchy display. IDK how best to fix.
        ani = FuncAnimation(fig, frame, np.arange(self.t,self.tmax,self.dt), 
                            interval=self.dt*1000/self.animSpeed, blit=True,
                           repeat=False)
        print("bouta save animation...")
        ani.save("particleAnimation.gif")
        plt.close()
        print("")
        print("finished animating!")

In [37]:
# TEST CELL FOR showAnimation1
seed = 42
rng = default_rng(seed=seed) # instatiate the default random number generator
sqrtNumParts = 2

pPosGrid = list(itertools.product(np.linspace(20, 180,sqrtNumParts), np.linspace(20,180,sqrtNumParts)))
pVelGrid = list(rng.random(size=(sqrtNumParts**2,2))*1) #slightly randomize initial velocities

particles = [Disk(pPosGrid[i], pVelGrid[i]) for i in range(sqrtNumParts**2)][:-1]

e1 = Expt(particles, dt=0.1, tmax=5, animSpeed=0.5)

print(e1.avgKE)
e1.showAnimation1("seed = " + str(seed) + "     ")
print(e1.avgKE)

1.4879215759371183


MovieWriter ffmpeg unavailable; using Pillow instead.


bouta save animation...
[████████████████████████████████]   
finished animating!
281.98810703491085


In [ ]:
# Fixing random state for reproducibility
np.random.seed(19680801)
# Fixing bin edges
HIST_BINS = np.linspace(-4, 4, 100)

# histogram our data with numpy
data = np.random.randn(1000)
n, _ = np.histogram(data, HIST_BINS)
def prepare_animation(bar_container):

    def animate(frame_number):
        # simulate new data coming in
        data = np.random.randn(1000)
        n, _ = np.histogram(data, HIST_BINS)
        for count, rect in zip(n, bar_container.patches):
            rect.set_height(count)
        return bar_container.patches
    return animate
# Output generated via `matplotlib.animation.Animation.to_jshtml`.

fig, ax = plt.subplots()
_, _, bar_container = ax.hist(data, HIST_BINS, lw=1,
                              ec="yellow", fc="green", alpha=0.5)
ax.set_ylim(top=55)  # set safe limit to ensure that all data is visible.

ani = FuncAnimation(fig, prepare_animation(bar_container), 50,
                              repeat=False, blit=True)
plt.show()

In [ ]:
KEs = [2*x**2 for x in range(5)]
fig, (ax1, ax2) = plt.subplots(1, 2)
ax2.hist(KEs, bins = 5, color = 'black', histtype = 'step', stacked = True);

In [ ]:
seed = 42
rng = default_rng(seed=seed) # instatiate the default random number generator
sqrtNumParts = 5

pPosGrid = list(itertools.product(np.linspace(20, 180,sqrtNumParts), np.linspace(20,180,sqrtNumParts)))
pVelGrid = list(rng.random(size=(sqrtNumParts**2,2))*1) #slightly randomize initial velocities

particles = [Disk(pPosGrid[i], pVelGrid[i]) for i in range(sqrtNumParts**2)][:-1]

e1 = Expt(particles, dt=0.1, tmax=15, animSpeed=0.5)

print(e1.avgKE)
e1.showAnimation("seed = " + str(seed) + "     ")
print(e1.avgKE)

In [ ]:
plt.close("all")

In [ ]:
#create and test an example disk
p = Disk([1,1], [6,8], 3, 2, 1)
p1 = Disk([3, 2], [3, 3], 1, 1, 1)

print(p < p1)
print(p == p1)
print(p > p1)
print(p.speed)
print(p.KE)
print(p.x)